In [ ]:
import os
import random
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, TimeSeriesSplit
from sklearn.linear_model import LinearRegression, HuberRegressor
from sklearn.metrics import mean_squared_error, r2_score
import xgboost as xgb
from glob import glob
from tqdm import tqdm
import tensorflow as tf

tf.logging.set_verbosity(tf.logging.ERROR)
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

from keras.layers import Input, Dense, LSTM, BatchNormalization, Flatten
from keras.models import Model
from keras.initializers import glorot_uniform
from keras.preprocessing.sequence import TimeseriesGenerator
from keras import optimizers, regularizers, activations
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, TensorBoard
import keras.backend as k

### Define a number of deep models as functions

#### LSTM model
- 2 LSTM layers
- Dense layer
- Batch Normalization layer

In [ ]:
def lstm1(input_shape_0, input_shape_1, activation, seed):
    input_tensor = Input(shape=(input_shape_0, input_shape_1),
                         dtype='float32',
                         name='input_tensor')

    lstm_1 = LSTM(units=32,
                  activation=activation,
                  dropout=0.2,
                  recurrent_dropout=0.2,
                  return_sequences=True,
                  kernel_initializer=glorot_uniform(random.seed(seed)),
                  name='lstm_1')(input_tensor)

    lstm_2 = LSTM(units=32,
                  activation=activation,
                  dropout=0.2,
                  recurrent_dropout=0.2,
                  return_sequences=False,
                  kernel_initializer=glorot_uniform(random.seed(seed)),
                  name='lstm_2')(lstm_1)

    dense = Dense(units=32,
                  activation=activation,
                  kernel_regularizer=l1(0.001),
                  kernel_initializer=glorot_uniform(random.seed(seed)),
                  name='dense')(lstm_2)

    batch_norm = BatchNormalization(axis=-1)(dense)

    # add a regression layer
    output_tensor = Dense(units=1,
                          activation=None,
                          name='output_tensor')(batch_norm)

    # specify input and output
    return Model(input_tensor, output_tensor)

#### NN model with 1 Dense layer

In [ ]:
def nn1(input_shape_0, input_shape_1, activation, seed, kernel_regularizer):
    input_tensor = Input(shape=(input_shape_0, input_shape_1),
                         dtype='float32',
                         name='input_tensor')

    dense_1 = Dense(units=32,
                    activation=activation,
                    kernel_regularizer=kernel_regularizer,
                    kernel_initializer=glorot_uniform(random.seed(seed)),
                    name='dense_1')(input_tensor)

    dense_1_flatten = Flatten(name='dense_1_flattened')(dense_1)

    # add a regression layer
    output_tensor = Dense(units=1,
                          activation=None,
                          name='output_tensor')(dense_1_flatten)

    # specify input and output
    return Model(input_tensor, output_tensor)

#### NN model with 2 Dense layers

In [ ]:
def nn2(input_shape_0, input_shape_1, activation, seed, kernel_regularizer):
    input_tensor = Input(shape=(input_shape_0, input_shape_1),
                         dtype='float32',
                         name='input_tensor')

    dense_1 = Dense(units=32,
                    activation=activation,
                    kernel_regularizer=kernel_regularizer,
                    kernel_initializer=glorot_uniform(random.seed(seed)),
                    name='dense_1')(input_tensor)

    dense_2 = Dense(units=16,
                    activation=activation,
                    kernel_regularizer=kernel_regularizer,
                    kernel_initializer=glorot_uniform(random.seed(seed)),
                    name='dense_2')(dense_1)

    dense_2_flatten = Flatten(name='dense_2_flatten')(dense_2)

    # add a regression layer
    output_tensor = Dense(units=1,
                          activation=None,
                          name='output_tensor')(dense_2_flatten)

    # specify input and output
    return Model(input_tensor, output_tensor)

#### NN model with 3 Dense layers

In [ ]:
def nn3(input_shape_0, input_shape_1, activation, seed, kernel_regularizer):
    input_tensor = Input(shape=(input_shape_0, input_shape_1),
                         dtype='float32',
                         name='input_tensor')

    dense_1 = Dense(units=32,
                    activation=activation,
                    kernel_regularizer=kernel_regularizer,
                    kernel_initializer=glorot_uniform(random.seed(seed)),
                    name='dense_1')(input_tensor)

    dense_2 = Dense(units=16,
                    activation=activation,
                    kernel_regularizer=kernel_regularizer,
                    kernel_initializer=glorot_uniform(random.seed(seed)),
                    name='dense_2')(dense_1)

    dense_3 = Dense(units=8,
                    activation=activation,
                    kernel_regularizer=kernel_regularizer,
                    kernel_initializer=glorot_uniform(random.seed(seed)),
                    name='dense_3')(dense_2)

    dense_3_flatten = Flatten(name='dense_2_flatten')(dense_3)

    # add a regression layer
    output_tensor = Dense(units=1,
                          activation=None,
                          name='output_tensor')(dense_3_flatten)

    # specify input and output
    return Model(input_tensor, output_tensor)

#### NN model with 4 Dense layers

In [ ]:
def nn4(input_shape_0, input_shape_1, activation, seed, kernel_regularizer):
    input_tensor = Input(shape=(input_shape_0, input_shape_1),
                         dtype='float32',
                         name='input_tensor')

    dense_1 = Dense(units=32,
                    activation=activation,
                    kernel_regularizer=kernel_regularizer,
                    kernel_initializer=glorot_uniform(random.seed(seed)),
                    name='dense_1')(input_tensor)

    dense_2 = Dense(units=16,
                    activation=activation,
                    kernel_regularizer=kernel_regularizer,
                    kernel_initializer=glorot_uniform(random.seed(seed)),
                    name='dense_2')(dense_1)

    dense_3 = Dense(units=8,
                    activation=activation,
                    kernel_regularizer=kernel_regularizer,
                    kernel_initializer=glorot_uniform(random.seed(seed)),
                    name='dense_3')(dense_2)

    dense_4 = Dense(units=4,
                    activation=activation,
                    kernel_regularizer=kernel_regularizer,
                    kernel_initializer=glorot_uniform(random.seed(seed)),
                    name='dense_4')(dense_3)

    dense_4_flatten = Flatten(name='dense_4_flatten')(dense_4)

    # add a regression layer
    output_tensor = Dense(units=1,
                          activation=None,
                          name='output_tensor')(dense_4_flatten)

    # specify input and output
    return Model(input_tensor, output_tensor)

#### NN model with 5 Dense layers

In [ ]:
def nn5(input_shape_0, input_shape_1, activation, seed, kernel_regularizer):
    input_tensor = Input(shape=(input_shape_0, input_shape_1),
                         dtype='float32',
                         name='input_tensor')

    dense_1 = Dense(units=32,
                    activation=activation,
                    kernel_regularizer=kernel_regularizer,
                    kernel_initializer=glorot_uniform(random.seed(seed)),
                    name='dense_1')(input_tensor)

    dense_2 = Dense(units=16,
                    activation=activation,
                    kernel_regularizer=kernel_regularizer,
                    kernel_initializer=glorot_uniform(random.seed(seed)),
                    name='dense_2')(dense_1)

    dense_3 = Dense(units=8,
                    activation=activation,
                    kernel_regularizer=kernel_regularizer,
                    kernel_initializer=glorot_uniform(random.seed(seed)),
                    name='dense_3')(dense_2)

    dense_4 = Dense(units=4,
                    activation=activation,
                    kernel_regularizer=kernel_regularizer,
                    kernel_initializer=glorot_uniform(random.seed(seed)),
                    name='dense_4')(dense_3)

    dense_5 = Dense(units=4,
                    activation=activation,
                    kernel_regularizer=kernel_regularizer,
                    kernel_initializer=glorot_uniform(random.seed(seed)),
                    name='dense_5')(dense_4)

    dense_5_flatten = Flatten(name='dense_5_flatten')(dense_5)

    # add a regression layer
    output_tensor = Dense(units=1,
                          activation=None,
                          name='output_tensor')(dense_5_flatten)

    # specify input and output
    return Model(input_tensor, output_tensor)

### Define a class for Neural Network

In [ ]:
class StocksNN:
    def __init__(self, features_file_path, period, lookback, step, features, target):
        """
        :param features_file_path: path to the csv file that includes the features
        :param period: daily or weekly
        :param lookback: number of time-steps in the past to use for predicting the next time-step
        :param step: number of steps for sampling
        :param features: a list of the features to use in the model
        :param target: return or excess_return
        """

        self.period = period
        self.lookback = lookback
        self.step = step
        self.target = target

        # make sure there's a sub directory to save results to
        self.save_in = os.path.join(os.getcwd(), 'results', f'{self.period}_features')
        if not os.path.exists(self.save_in):
            os.makedirs(self.save_in)

        # concatenate a unique string for this model
        file_name = os.path.basename(features_file_path)
        file_root, file_ext = os.path.splitext(file_name)
        self.unique_string = '_'.join(file_root.lower().split())

        # ------------------------------------------------- READ DATA
        # read data and choose only features wanted for this model from the multi-indexed header
        self.df = pd.read_csv(features_file_path, index_col=[0], header=[0, 1], parse_dates=True)
        self.df = self.df[features].droplevel(level=0, axis=1)

        # drop either return or excess_return
        if self.target == 'return':
            self.df = self.df.drop('excess_return', axis=1)
        elif self.target == 'excess_return':
            self.df = self.df.drop('return', axis=1)        

        # ------------------------------------------------- SELECT DATA INDICES FOR SPLITTING
        # split data 80/20 for training/testing without shuffling to keep time order
        # keep target as a feature in data
        # the initial training data will be split later into n splits
        self.X_train_init, self.X_test_init = train_test_split(self.df, test_size=0.20, shuffle=False)        

        # select a number of splits equals to the number of years in training data
        self.n_split = len(set(self.X_train_init.index.year))

        # ---------------------------------------------------- DATA SCALING
        # fit scaler to training data only (including target as it will be a feature as well)
        # select target
        self.scaler = StandardScaler()
        self.X_train_init = self.scaler.fit_transform(self.X_train_init)
        self.y_train_init = self.X_train_init[:, -1]

        # transform test data (including target as it will be a feature as well)
        # select target
        self.X_test = self.scaler.transform(self.X_test_init)
        self.y_test = self.X_test[:, -1]

    # ------------------------------------------------- BUILD & COMPILE MODEL
    def build_and_compile_model(self, model_function, model_name, activation, optimizer, seed, kernel_regularizer):
        self.input_shape_0 = self.lookback // self.step
        self.input_shape_1 = self.X_train_init.shape[1]

        self.model = model_function(self.input_shape_0, self.input_shape_1, activation, seed, kernel_regularizer)

        self.model.compile(optimizer=optimizer,
                           loss='mse')
        
        # append unique string
        self.unique_string = f'{self.unique_string}_{model_name}_{round(k.eval(self.model.optimizer.lr), 3)}'

    # ------------------------------------------------- WALK FORWARD VALIDATION
    def walk_forward_validation(self, batch_size, epochs, n_splits=None):

        self.batch_size = batch_size

        # if n_splits is not provided, use number of years in training data as above
        if n_splits is None:
            n_splits = self.n_split

        # split initial training data for training/validation
        # forward walking validation
        tscv = TimeSeriesSplit(n_splits=n_splits)
        eval_scores = list()
        for train_index, val_index in tscv.split(self.X_train_init):
            X_train, X_val = self.X_train_init[train_index], self.X_train_init[val_index]
            y_train, y_val = self.y_train_init[train_index], self.y_train_init[val_index]            

            # ----------------------------------------- CREATE GENERATORS
            # generator output is a list of batches
            # each batch is a tuple of (samples, targets)
            train_gen = TimeseriesGenerator(X_train,
                                            y_train,
                                            length=self.lookback,
                                            sampling_rate=self.step,
                                            stride=1,
                                            batch_size=self.batch_size)            

            val_gen = TimeseriesGenerator(X_val,
                                          y_val,
                                          length=self.lookback,
                                          sampling_rate=self.step,
                                          stride=1,
                                          batch_size=self.batch_size)
            
            # ----------------------------------------- FIT & EVALUATE            
            # interrupt training when improvement stops
            # continually save the model during training (only current best)
            # reduce learning rate when the validation loss has stopped improving
            callbacks_list = [EarlyStopping(monitor='val_loss',
                                            patience=5),
                              ModelCheckpoint(filepath=os.path.join(self.save_in, f'{self.unique_string}.h5'),
                                              monitor='val_loss',
                                              save_best_only=True),
                              ReduceLROnPlateau(monitor='val_loss',
                                                factor=0.1,
                                                patience=10)
                             ]

            self.model.fit_generator(train_gen,
                                     steps_per_epoch=len(train_gen),
                                     epochs=epochs,
                                     validation_data=val_gen,
                                     validation_steps=len(val_gen),
                                     callbacks=callbacks_list,
                                     verbose=False)

            # get eval metric and append to list
            eval_score = self.model.evaluate_generator(val_gen, steps=len(val_gen))
            eval_scores.append(eval_score)

        # average evaluation scores
        return pd.Series(np.mean(eval_scores), index=['mse'])

    def predict_on_test_data_nn(self):
        self.test_gen = TimeseriesGenerator(self.X_test,
                                            self.y_test,
                                            length=self.lookback,
                                            sampling_rate=self.step,
                                            stride=1,
                                            batch_size=self.batch_size)        

        self.predictions = self.model.predict_generator(self.test_gen,
                                                        steps=len(self.test_gen))

        # inverse transform predictions to un-normalize
        # first, repeat as many times as the transformer expects
        self.predictions = np.repeat(self.predictions, self.df.shape[1], axis=1)
        self.predictions = self.scaler.inverse_transform(self.predictions)[:, -1]

        # get test data using the generator to get corresponding targets
        # each batch is a tuple of numpy arrays (samples, targets), get the targets as a list comprehension
        # convert list of numpy arrays into a numpy vector, reshape into 1D array
        self.true = np.concatenate([batch[1] for batch in self.test_gen], axis=0).reshape(-1, 1)

        # inverse transform targets to un-normalize
        # first, repeat as many times as the transformer expects
        self.true = np.repeat(self.true, self.df.shape[1], axis=1)
        self.true = self.scaler.inverse_transform(self.true)[:, -1]

        # index was lost when scaling. Get the index from self.X_test_init
        # however, we will loose few of the last samples in self.test_gen.
        # use the length of self.true to get the exact number of test samples used
        return pd.DataFrame({'true': self.true, 'predictions': self.predictions},
                            index=self.X_test_init.index[:self.true.shape[0]])

    def predict_on_test_data_lewellen(self, n_splits=None):
        
        df_size_factors = None
        
        # if n_splits is not provided, use number of years in training data as above
        if n_splits is None:
            n_splits = self.n_split
        
        if self.period == 'daily':
            df_size_factors = pd.read_csv(os.path.join(os.getcwd(), 'data', 'Daily_F_F_Research_Data_Factors.csv'),
                                          index_col=0,
                                          parse_dates=True)
        elif self.period == 'weekly':
            df_size_factors = pd.read_csv(os.path.join(os.getcwd(), 'data', 'Weekly_F_F_Research_Data_Factors.csv'),
                                          index_col=0,
                                          parse_dates=True).resample('W').last()

        # select only indexes in stocks
        df_size_factors = df_size_factors[df_size_factors.index.isin(self.df.index)]

        # select target and shift by 1 time-step and drop first row
        # concatenate features and remove first row
        y = self.df[self.target].shift(self.lookback).dropna()
        X = pd.concat([df_size_factors['SMB'], self.df['df_book_to_market'], self.df['momentum_1M']],
                      axis=1, join='inner').iloc[self.lookback:]

        # rows should have the same order as the first train_test_split
        X_train, X_test, y_train, true_lewellen = train_test_split(X, y, test_size=0.20, shuffle=False)
        
        # fit scaler to training data only
        # transform test data
        scaler = StandardScaler()
        X_train = scaler.fit_transform(X_train)
        X_test = scaler.transform(X_test)
        
        # create a Huber regressor        
        model = HuberRegressor(alpha=0.0001)

        # split initial training data for training/validation
        # forward walking validation
        tscv = TimeSeriesSplit(n_splits=n_splits)
        eval_scores = list()
        for train_index, val_index in tscv.split(self.X_train_init):
            X_train, X_val = self.X_train_init[train_index], self.X_train_init[val_index]
            y_train, y_val = self.y_train_init[train_index], self.y_train_init[val_index]            

            # fit Huber regressor        
            model = HuberRegressor(alpha=0.0001)
            model.fit(X_train, y_train)

        # predict on test data
        predictions_lewellen = model.predict(X_test)

        # create df with all time-steps. Select only indexes that are in the self.X_test_init
        df = pd.DataFrame({'true': true_lewellen, 'predictions': predictions_lewellen})
        
        return df[df.index.isin(self.X_test_init.index[:self.true.shape[0]])]

### Define a class for XGBoost

In [ ]:
class StocksXGB:
    def __init__(self, features_file_path, period, lookback, step, features, target):
        """
        :param features_file_path: path to the csv file that includes the features
        :param period: daily or weekly
        :param lookback: number of time-steps in the past to use for predicting the next time-step
        :param step: number of steps for sampling
        :param features: a list of the features to use in the model
        :param target: return or excess_return
        """
        
        self.period = period
        self.lookback = lookback
        self.step = step
        self.target = target

        # make sure there's a sub directory to save results to
        self.save_in = os.path.join(os.getcwd(), 'results', f'{self.period}_features')
        if not os.path.exists(self.save_in):
            os.makedirs(self.save_in)

        # concatenate a unique string for this model
        file_name = os.path.basename(features_file_path)
        file_root, file_ext = os.path.splitext(file_name)
        self.unique_string = '_'.join(file_root.lower().split())

        # ------------------------------------------------- READ DATA
        # read data and choose only features wanted for this model from the multi-indexed header
        self.df = pd.read_csv(features_file_path, index_col=[0], header=[0, 1], parse_dates=True)
        self.df = self.df[features].droplevel(level=0, axis=1)

        # drop either return or excess_return
        if self.target == 'return':
            self.df = self.df.drop('excess_return', axis=1)
        elif self.target == 'excess_return':
            self.df = self.df.drop('return', axis=1)

        # ------------------------------------------------- FEATURES EXTRACTIONS
        # extract features to infer time
        # this's important in xgb as it selects samples randomly so time-order is lost
        self.df['month'] = self.df.index.month
        self.df['quarter'] = self.df.index.quarter
        self.df['year'] = self.df.index.year

        if self.period == 'daily':
            self.df['dayofyear'] = self.df.index.dayofyear
            self.df['dayofmonth'] = self.df.index.day
        elif self.period == 'weekly':
            self.df['weekofyear'] = self.df.index.weekofyear

        # shift return by lookback so the target is not the current return but in the future
        # this is similar to what the keras generator does for the NN model
        # the old return will be used as a normal feature now (similar to the NN model as well)
        self.df['return_future'] = self.df[self.target].shift(self.lookback)

        # ------------------------------------------------- SPLIT DATA
        # split data 80/20 for training/testing without shuffling to keep time order
        # keep target as a feature in data
        # the initial training data will be split later into n splits
        self.X_train_init, self.X_test_init = train_test_split(self.df.dropna(), test_size=0.20, shuffle=False)

        # select a number of splits equals to the number of years in training data
        self.n_split = len(set(self.X_train_init.index.year))

        # trees don't require scaling or centering of data
        self.y_train_init = self.X_train_init['return_future'].values
        self.X_train_init = self.X_train_init.drop(['return_future'], axis=1).values

        self.y_test = self.X_test_init['return_future'].values
        self.X_test = self.X_test_init.drop(['return_future'], axis=1).values

    # ------------------------------------------------- BUILD & COMPILE MODEL
    def build_model(self, objective, max_depth, learning_rate, n_estimators, seed):
        self.max_depth = max_depth
        self.learning_rate = learning_rate
        self.n_estimators = n_estimators

        self.xgb_model = xgb.XGBRegressor(objective=objective,
                                          max_depth=self.max_depth,
                                          learning_rate=self.learning_rate,
                                          n_estimators=self.n_estimators,
                                          random_state=seed)

        # append unique string
        self.unique_string = f"{self.unique_string}_xgb_{self.max_depth}_{round(self.learning_rate, 3)}_{self.n_estimators}"

    # ------------------------------------------------- WALK FORWARD VALIDATION
    def walk_forward_validation(self, n_splits=None):

        # if n_splits is not provided, use number of years in training data as above
        if n_splits is None:
            n_splits = self.n_split

        # split initial training data for training/validation
        # forward walking validation
        tscv = TimeSeriesSplit(n_splits=n_splits)
        eval_scores = list()
        for train_index, val_index in tscv.split(self.X_train_init):
            X_train, X_val = self.X_train_init[train_index], self.X_train_init[val_index]
            y_train, y_val = self.y_train_init[train_index], self.y_train_init[val_index]
            # print(f'X_train: {len(X_train)}', f'X_val: {len(X_val)}')

            # ----------------------------------------- FIT & EVALUATE
            self.xgb_model.fit(X_train,
                               y_train.ravel(),
                               eval_set=[(X_val, y_val)],
                               eval_metric=['rmse'],
                               verbose=False)

            # get the last tree eval metrics and append to list
            # square rmse to get mse
            eval_score = self.xgb_model.evals_result_['validation_0']['rmse'][-1] ** 2
            eval_scores.append(eval_score)

            # average evaluation scores
            return pd.Series(np.mean(eval_scores), index=['mse'])

    def predict_on_test_data(self):
        predictions = self.xgb_model.predict(self.X_test)

        # index was lost when scaling. Get the index from self.X_test_init
        return pd.DataFrame({'true': self.y_test, 'predictions': predictions}, index=self.X_test_init.index)

### Test Results: scores and plots

#### Calculate predicted vs true scores for test data

In [ ]:
def calc_predictions_avg_test_scores(true, predictions):
    """
    Calculate test scores between true and predicted returns
    :param true: true returns
    :param predictions: predicted returns
    :return: Pandas Series
    """
    r2_score_modified = 1 - (np.sum((true - predictions) ** 2)) / (np.sum(true ** 2))
    r2_score_orig = r2_score(true, predictions)
    mse = mean_squared_error(true, predictions)
    sse = np.sum((predictions - true) ** 2)

    return pd.Series([r2_score_modified, r2_score_orig, mse, sse],
                     index=['r2_modified', 'r2_original', 'mse', 'sse'])

#### Plot predicted vs true returns for test data

In [ ]:
def plot_predictions_avg_stock(df_models_avg_true_best, df_models_avg_pre_best, df_benchmarks, save_in, unique_string):
    """
    Plot the averaged predicted returns vs true for model, and lewellen per stock
    :param df_predictions_avg: Pandas DataFrame
    :param save_in: directory to save figure
    :param unique_string: to add to the figure name
    :return:
    """
    start_time = df_models_avg_true_best.index[0]
    end_time = df_models_avg_true_best.index[-1]
    stocks = set(df_models_avg_true_best.columns.get_level_values(1))

    custom_lines = [Line2D([0], [0], color='b', lw=2),
                    Line2D([0], [0], color='r', lw=2)]

    # loop over unique stocks found in the second level
    for stock in stocks:
        fig, ax = plt.subplots(2, 2, figsize=(12, 6))
        fig.suptitle(f'{stock} avg. Predictions', fontsize=12)

        ax[0, 0].plot(df_models_avg_true_best['NN', stock], 'b', label='Actual')
        ax[0, 0].plot(df_models_avg_pre_best['NN', stock], 'r', label='Predicted')
        ax[0, 0].set_xticks([ax[0, 0].get_xticks()[0], ax[0, 0].get_xticks()[-1]], minor=False)
        ax[0, 0].set_xticklabels([start_time.strftime("%b %Y"), end_time.strftime("%b %Y")])
        ax[0, 0].set_title(f"NN, lr={df_models_avg_true_best['NN', stock].columns.values}", fontsize=10)

        ax[0, 1].plot(df_models_avg_true_best['XGB', stock], 'b', label='Actual')
        ax[0, 1].plot(df_models_avg_pre_best['XGB', stock], 'r', label='Predicted')
        ax[0, 1].set_xticks([ax[0, 1].get_xticks()[0], ax[0, 1].get_xticks()[-1]], minor=False)
        ax[0, 1].set_xticklabels([start_time.strftime("%b %Y"), end_time.strftime("%b %Y")])
        ax[0, 1].set_title(f"XGBoost, lr={df_models_avg_true_best['XGB', stock].columns.values}", fontsize=10)

        ax[1, 0].plot(df_benchmarks['lewellen', stock, 'true'], 'b', label='Actual')
        ax[1, 0].plot(df_benchmarks['lewellen', stock, 'predictions'], 'r', label='Predicted')
        ax[1, 0].set_xticks([ax[1, 0].get_xticks()[0], ax[1, 0].get_xticks()[-1]], minor=False)
        ax[1, 0].set_xticklabels([start_time.strftime("%b %Y"), end_time.strftime("%b %Y")])
        ax[1, 0].set_title('Lewellen', fontsize=10)

        ax[1, 1].legend(custom_lines, ['Actual', 'Predicted'], loc='center')
        ax[1, 1].axis('off')

        plt.savefig(os.path.join(save_in, f'{unique_string}_predictions_avg_{stock}.png'),
                    orientation='portrait',
                    format='png')
        plt.close()

In [ ]:
def plot_predictions_avg_all_stocks(df_models_avg_true_best, df_models_avg_pre_best, df_benchmarks, save_in, unique_string):
    """
    Plot the averaged predicted returns vs true for model, and lewellen for all stocks
    :param df_predictions_avg: Pandas DataFrame
    :param save_in: directory to save figure
    :param unique_string: to add to the figure name
    :return:
    """
    start_time = df_models_avg_true_best.index[0]
    end_time = df_models_avg_true_best.index[-1]
    stocks = set(df_models_avg_true_best.columns.get_level_values(1))

    fig, ax = plt.subplots(2, 2, figsize=(12, 6))
    fig.suptitle('Avg. Predictions all Stocks', fontsize=12)

    ax[0, 0].set_title('NN', fontsize=10)
    ax[0, 1].set_title('XGBoost', fontsize=10)
    ax[1, 0].set_title('Lewellen', fontsize=10)

    # loop over unique stocks found in the second level
    for stock in stocks:
        ax[0, 0].plot(df_models_avg_true_best['NN', stock], 'b', label='Actual')
        ax[0, 0].plot(df_models_avg_pre_best['NN', stock], 'r', label='Predicted')

        ax[0, 1].plot(df_models_avg_true_best['XGB', stock], 'b', label='Actual')
        ax[0, 1].plot(df_models_avg_pre_best['XGB', stock], 'r', label='Predicted')

        ax[1, 0].plot(df_benchmarks['lewellen', stock, 'true'], 'b', label='Actual')
        ax[1, 0].plot(df_benchmarks['lewellen', stock, 'predictions'], 'r', label='Predicted')

    ax[0, 0].set_xticks([ax[0, 0].get_xticks()[0], ax[0, 0].get_xticks()[-1]], minor=False)
    ax[0, 0].set_xticklabels([start_time.strftime("%b %Y"), end_time.strftime("%b %Y")])

    ax[1, 0].set_xticks([ax[1, 0].get_xticks()[0], ax[1, 0].get_xticks()[-1]], minor=False)
    ax[1, 0].set_xticklabels([start_time.strftime("%b %Y"), end_time.strftime("%b %Y")])

    ax[0, 1].set_xticks([ax[0, 1].get_xticks()[0], ax[0, 1].get_xticks()[-1]], minor=False)
    ax[0, 1].set_xticklabels([start_time.strftime("%b %Y"), end_time.strftime("%b %Y")])

    # add fake line to add one legend only
    custom_lines = [Line2D([0], [0], color='b', lw=2),
                    Line2D([0], [0], color='r', lw=2)]

    ax[1, 1].legend(custom_lines, ['Actual', 'Predicted'], loc='center')
    ax[1, 1].axis('off')

    plt.savefig(os.path.join(save_in, f'{unique_string}_predictions_avg_all_stocks.png'),
                orientation='portrait',
                format='png')
    plt.close()

### Main script
- Try 2 seeds and 3 learning rates (i.e. 6 combinations)

- For each combination:
    - Fit nn model (lstm1, nn1, nn2, nn3, nn4, nn5)    
    - Get walk forward validation evaluation metrics
    - Make predictions on test data
    - Fit XGBoost model
    - Get walk forward validation evaluation metrics
    - Make predictions on test data
    - Make predictions on test data using Lewellen benchmark

In [ ]:
stocks_csv_dir_name = os.path.join(os.getcwd(), 'data', 'weekly_features')
stocks_csv_files_full_path_list = glob(os.path.join(stocks_csv_dir_name, '*.csv'))

features = ['stock_features', 'macro_features', 'momentum_features', 'annual_features', 'tensor_product', 'returns']

period = 'weekly'
lookback = 12
step = 1
seeds = [1, 42]

learning_rates = [0.001, 0.005, 0.01]

save_in = os.path.join(os.getcwd(), 'results', f'{period}_features')

# ------------------------------------------------ NN ARGS
model_function = nn5
model_name = 'nn5'

activation = activations.relu
kernel_regularizer = regularizers.l1(0.001)

batch_size = 32
epochs = 100
unique_string = f'{model_name}_relu_adam_{epochs}'

# ------------------------------------------------ XGB ARGS
objective = 'reg:squarederror'
max_depth = 2
n_estimators = 1000

unique_string = f"{unique_string}_xgb_{max_depth}_{n_estimators}"

# ------------------------------------------------------------------------------------------------------------------
# create empty MultiIndexed DataFrames to save all results for all stocks
df_models = pd.DataFrame(columns=pd.MultiIndex(levels=[[], [], [], [], []],
                                               codes=[[], [], [], [], []],
                                               names=['model', 'stock', 'lr', 'seed', 'true_pre']))
df_benchmarks = pd.DataFrame(columns=pd.MultiIndex(levels=[[], [], []],
                                                   codes=[[], [], []],
                                                   names=['benchmark', 'stock', 'true_pre']))
df_models_eval_metrics = pd.DataFrame(columns=pd.MultiIndex(levels=[[], [], []],
                                                            codes=[[], [], []],
                                                            names=['data', 'stock', 'lr']))
df_models_avg_best_all_stocks = pd.DataFrame()

# loop over each stock
# for NN and XGB, loop over each learning rate and seed
# for lewellen, no need to loop over learning rates and seeds
# train a new model, get predictions, then save results in the master DataFrame
for stocks_csv_file_full_path in tqdm(stocks_csv_files_full_path_list):
        file_name = os.path.basename(stocks_csv_file_full_path)
        file_root, _ = os.path.splitext(file_name)
        print(f'\nProcessing {file_name}...')

        # loop over learning rates
        for learning_rate in tqdm(learning_rates):
            print(f'\nlearning rate {learning_rate}...')
            optimizer = optimizers.adam(lr=learning_rate)

            # loop over seeds
            for seed in tqdm(seeds):
                print(f'\nseed {seed}...')

                # -------------------------------------------- NN MODEL --------------------------------------------
                print('\nFitting NN model...')
                # create NN object
                stock_nn = StocksNN(stocks_csv_file_full_path,
                                    period,
                                    lookback,
                                    step,
                                    features,
                                    'excess_return')
                stock_nn.build_and_compile_model(model_function,
                                                 model_name,
                                                 activation,
                                                 optimizer,
                                                 seed,
                                                 kernel_regularizer)

                df_models_eval_metrics['NN', f'{file_root}', learning_rate] = stock_nn.walk_forward_validation(
                    batch_size, epochs)

                df_nn_predictions = stock_nn.predict_on_test_data_nn()
                df_models['NN', f'{file_root}', learning_rate, seed, 'true'] = df_nn_predictions['true']
                df_models['NN', f'{file_root}', learning_rate, seed, 'predictions'] = df_nn_predictions['predictions']

                # -------------------------------------------- XGB MODEL -------------------------------------------
                print('\nFitting XGB model...')
                # create XGB object
                stock_xgb = StocksXGB(stocks_csv_file_full_path,
                                      period,
                                      lookback,
                                      step,
                                      features,
                                      'excess_return')
                stock_xgb.build_model(objective,
                                      max_depth,
                                      learning_rate,
                                      n_estimators,
                                      seed)

                df_models_eval_metrics['XGB', f'{file_root}', learning_rate] = stock_xgb.walk_forward_validation()

                df_xgb_predictions = stock_xgb.predict_on_test_data()
                df_models['XGB', f'{file_root}', learning_rate, seed, 'true'] = df_xgb_predictions['true']
                df_models['XGB', f'{file_root}', learning_rate, seed, 'predictions'] = df_xgb_predictions[
                    'predictions']

        # ------------------------------------------------ LEWELLEN
        print('Predicting lewellen...')
        df_lewellen_pre = stock_nn.predict_on_test_data_lewellen()
        df_benchmarks['lewellen', f'{file_root}', 'true'] = df_lewellen_pre['true']
        df_benchmarks['lewellen', f'{file_root}', 'predictions'] = df_lewellen_pre['predictions']

- Average predictions across seeds
- Choose learning rate with the best evaluation scores on the validation set
- Calculate evaluation scores on the test set (using chosen learning rate)
- Save as csv files
- Plot results

In [ ]:
# --------------------------------------------------- BENCHMARK ----------------------------------------------------
# save predictions
df_benchmarks.to_csv(os.path.join(save_in, f'{unique_string}_benchmarks.csv'))

# calculate and save test scores
df_benchmarks_scores = df_benchmarks.dropna().groupby(level=[0, 1], axis=1).apply(
    lambda s: calc_predictions_avg_test_scores(s.iloc[:, 0], s.iloc[:, 1]))
df_benchmarks_scores.to_csv(os.path.join(save_in, f'{unique_string}_benchmarks_scores.csv'))

# ----------------------------------------------------- MODELS -----------------------------------------------------
# save each stock predictions
df_models = df_models.dropna(axis=0)
df_models.to_csv(os.path.join(save_in, f'{unique_string}_models.csv'))

# average per stock predictions across all seeds
df_models_avg = df_models.groupby(level=[0, 1, 2, 4], axis=1).mean()
df_models_avg.to_csv(os.path.join(save_in, f'{unique_string}_models_avg.csv'))

# calculate test scores per stock averaged predictions
df_models_avg_scores = df_models_avg.groupby(level=[0, 1, 2], axis=1).apply(
    lambda s: calc_predictions_avg_test_scores(s.iloc[:, 0], s.iloc[:, 1]))
df_models_avg_scores.to_csv(os.path.join(save_in, f'{unique_string}_models_avg_scores.csv'))

# select best learning rate per stock (minimum averaged 'mse' on validation set)
df_models_eval_metrics_best = df_models_eval_metrics[
    df_models_eval_metrics.groupby(level=[0, 1], axis=1).idxmin().loc['mse']]
df_models_eval_metrics_best.to_csv(os.path.join(save_in, f'{unique_string}_models_eval_metrics_best.csv'))

# select metrics for best learning rate per stock (using previous best learning rates on validation)
df_models_avg_scores_best = df_models_avg_scores[df_models_eval_metrics_best.columns]
df_models_avg_scores_best.to_csv(os.path.join(save_in, f'{unique_string}_models_avg_scores_best.csv'))

# select true/pre values for best learning rate per stock (using previous best learning rates on validation)
# separate true from predictions to get rid of the forth level
df_models_avg_true = df_models_avg.loc[:, (slice(None), slice(None), slice(None), 'true')]
df_models_avg_true.columns = df_models_avg_true.columns.droplevel(3)
df_models_avg_true_best = df_models_avg_true[df_models_eval_metrics_best.columns]
df_models_avg_true_best.to_csv(os.path.join(save_in, f'{unique_string}_models_avg_true_best.csv'))

df_models_avg_pre = df_models_avg.loc[:, (slice(None), slice(None), slice(None), 'predictions')]
df_models_avg_pre.columns = df_models_avg_pre.columns.droplevel(3)
df_models_avg_pre_best = df_models_avg_pre[df_models_eval_metrics_best.columns]
df_models_avg_pre_best.to_csv(os.path.join(save_in, f'{unique_string}_models_avg_pre_best.csv'))

# plot per stock
plot_predictions_avg_stock(df_models_avg_true_best,
                           df_models_avg_pre_best,
                           df_benchmarks,
                           save_in,
                           unique_string)
# plot all stocks
plot_predictions_avg_all_stocks(df_models_avg_true_best,
                           df_models_avg_pre_best,
                           df_benchmarks,
                           save_in,
                           unique_string)

# concatenate averaged predictions for all stocks vertically
# get rid of the third level
df_models_avg_true_best.columns = df_models_avg_true_best.columns.droplevel(2)
df_models_avg_true_best_all_stocks = df_models_avg_true_best.groupby(level=[0], axis=1).apply(
    lambda df: df.values.flatten())

df_models_avg_pre_best.columns = df_models_avg_pre_best.columns.droplevel(2)
df_models_avg_pre_best_all_stocks = df_models_avg_pre_best.groupby(level=[0], axis=1).apply(
    lambda df: df.values.flatten())

df_models_avg_best_all_stocks['NN'] = calc_predictions_avg_test_scores(
    df_models_avg_true_best_all_stocks.loc['NN'],
    df_models_avg_pre_best_all_stocks.loc['NN'])

df_models_avg_best_all_stocks['XGB'] = calc_predictions_avg_test_scores(
    df_models_avg_true_best_all_stocks.loc['XGB'],
    df_models_avg_pre_best_all_stocks.loc['XGB'])

df_benchmarks_all_stocks = df_benchmarks.groupby(level=[0, 2], axis=1).apply(
    lambda df: df.values.flatten())

df_models_avg_best_all_stocks['lewellen'] = calc_predictions_avg_test_scores(
    df_benchmarks_all_stocks.loc['lewellen', 'true'],
    df_benchmarks_all_stocks.loc['lewellen', 'predictions'])

df_models_avg_best_all_stocks.to_csv(os.path.join(save_in, f'{unique_string}_models_avg_best_all_stocks.csv'))